In [88]:
import gzip
import os
import random
import numpy as np

## paths to files. Do not change this
simInputFile = "Q1/word-similarity-dataset"
analogyInputFile = "analogy_dataset.txt"
vectorgzipFile = "Q1/glove.6B.300d.txt.gz"
vectorTxtFile = "Q1/glove.6B.300d.txt"   # If you extract and use the gz file, use this.
analogyTrainPath = "Q1/wordRep/"
simOutputFile = "Q1/simOutput.csv"
simSummaryFile = "Q1/simSummary.csv"
anaSOln = "Q1/analogySolution.csv"
Q4List = "Q4/wordList.csv"


In [89]:
def readFiles(filename):
    #f=open("analogy_dataset.txt",w)
    f = open(filename,"r")
    lines = f.readlines()
    content =[]
    for x in lines:
        x = x.rstrip()
        if(x != ''):
            content.append(x.strip())
    f.close()
    return content

In [90]:
def checkInDict(wordPair,validateVectors):
    wordPair.replace('\t',' ')
    words = wordPair.split()
    for word in words:
        if(word not in validateVectors):
            return 0
    return 1
    

In [91]:
def generateData(validateVectors):
    files = os.listdir("./Q1/wordRep/wordnet")
    random.seed(100)
    fout = open("./analogy_dataset.txt","w")
    for f in files:
        list1= readFiles("./Q1/wordRep/wordnet/"+f)
        list2 = []
        for f1 in files:
            if f == f1:
                continue
            else:
                l =readFiles("./Q1/wordRep/wordnet/"+f1)
                list2 = list2 + l
        for word in list1:
            query =[None] *7
            if(checkInDict(word,validateVectors) == 0):
                continue
            query[0] = word
            opt = random.randint(0,len(list1)-1)
            while (checkInDict(list1[opt],validateVectors) == 0):
                opt = random.randint(0,len(list1)-1)
            pos = random.randint(1,5)
            query[pos] = list1[opt]
            query[6] = chr(ord('a') + pos-1)
            for i in range(1,6):
                if(i == pos): continue
                else:
                    option = random.randint(0,len(list2)-1)
                    while (checkInDict(list2[option],validateVectors) == 0):
                        option = random.randint(0,len(list2)-1)
                    query[i] = list2[option]
            for i in range(0,7):
                fout.flush()
                query[i] = query[i].replace('\t',' ')
                fout.write(query[i]+'\n')
            fout.write('\n\n')
    fout.close()
            #print(query[6])

In [92]:
# Similarity Dataset
simDataset = [item.split(" | ") for item in open(simInputFile).read().splitlines()]
# Analogy dataset
analogyDataset = [[stuff.strip() for stuff in item.strip('\n').split('\n')] for item in open(analogyInputFile).read().split('\n\n')]

def vectorExtract(simD = simDataset, anaD = analogyDataset, vect = vectorgzipFile):
    simList = [stuff for item in simD for stuff in item]
    analogyList = [thing for item in anaD for stuff in item[0:4] for thing in stuff.split()]
    simList.extend(analogyList)
    wordList = set(simList)
    print(len(wordList))
    wordDict = dict()
    
    vectorFile = gzip.open(vect, 'r')
    for line in vectorFile:
        if line.split()[0].strip().decode("utf-8") in wordList:
            wordDict[line.split()[0].strip().decode("utf-8")] = line.split()[1:]
    
    
    vectorFile.close()
    print('retrieved', len(wordDict.keys()))
    return wordDict

# Extracting Vectors from Analogy and Similarity Dataset
#validateVectors = vectorExtract()

In [112]:
def generateInputMat(validateVectors):
    fin = open("analogy_dataset.txt","r")
    x = np.zeros((1,3600))
    y = np.zeros((1,1))
    trainX =[]
    trainY =[]
    line = fin.readline()
    while (line != ''):
        not_in_dict = 0
        tempvec =[]
        for i in range(0,6):
            l = line.rstrip().split()
            for j in range(0,2):
                if (l[j] in validateVectors):
                    tempvec = tempvec + validateVectors[l[j]]
                else:
                    not_in_dict =1
            line = fin.readline()
        tempy =line.rstrip()
        line = fin.readline()
        line = fin.readline()
        line = fin.readline()
        if (not_in_dict == 0):
            #x = np.vstack((x,tempvec))
            trainX.append(tempvec)
            tempy = ord(tempy)-ord('a')
            yvec= [0,0,0,0,0]
            yvec[tempy] =1
            #y = np.vstack((y,tempy))
            trainY.append(yvec)
        
    fin.close()
    trainX = np.array(trainX)
    trainY = np.array(trainY)
    return trainX[1:],trainX[1:],trainY[1:],trainY[1:]

In [110]:
# Requires: numpy, sklearn>=0.18.1, tensorflow>=1.0

# NOTE: In order to make the code simple, we rewrite x * W_1 + b_1 = x' * W_1'
# where x' = [x | 1] and W_1' is the matrix W_1 appended with a new row with elements b_1's.
# Similarly, for h * W_2 + b_2
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)


def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

def get_iris_data():
    """ Read the iris data set and split them into training and test sets """
    iris   = datasets.load_iris()
    data   = iris["data"]
    target = iris["target"]

    # Prepend the column of 1s for bias
    N, M  = data.shape
    all_X = np.ones((N, M + 1))
    all_X[:, 1:] = data

    # Convert into one-hot vectors
    num_labels = len(np.unique(target))
    all_Y = np.eye(num_labels)[target]  # One liner trick!
    return train_test_split(all_X, all_Y, test_size=0.33, random_state=RANDOM_SEED)

def trainnet(validateVectors):
    train_X, test_X, train_y, test_y = generateInputMat(validateVectors)

    # Layer's sizes
    x_size = train_X.shape[1]   # Number of input nodes: 4 features and 1 bias
    h_size = 256                # Number of hidden nodes
    y_size = train_y.shape[1]   # Number of outcomes (3 iris flowers)

    # Symbols
    X = tf.placeholder("float", shape=[None, x_size])
    y = tf.placeholder("float", shape=[None, y_size])

    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    for epoch in range(10):
        # Train with each example
        for i in range(len(train_X)):
            sess.run(updates, feed_dict={X: train_X[i: i + 1], y: train_y[i: i + 1]})

        train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: train_X, y: train_y}))
        test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: test_X, y: test_y}))

        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

    sess.close()



In [ ]:
trainnet(validateVectors)

Epoch = 1, train accuracy = 25.17%, test accuracy = 25.17%


In [108]:
#generateData(validateVectors)
#validateVectors = vectorExtract()
trainX,trainY,testX,testY = generateInputMat(validateVectors)
print(trainX.shape)

(7146, 3600)
